In [1]:
#@title Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_datareader import data as pdr
from datetime import datetime
import yfinance as yf
import os

In [2]:
#@title fredapi

from fredapi import Fred

fred_key = "9d0f00f8cad2d867849f09410fb483a0"
fred = Fred(api_key=fred_key)

In [3]:
#@title Macro variables

start = "2000-01-01"
end = "2024-12-31"

fred_series = {
    "CPI": "CPIAUCSL",
    "Unemployment": "UNRATE",
    "10Y_Treasury": "GS10",
    "FedFundsRate": "FEDFUNDS",
    "IP_Index": "INDPRO"
}

In [4]:
#@title Download fred data

fred_df = pd.DataFrame()

for label, code in fred_series.items():
    fred_df[label] = fred.get_series(code, observation_start=start)

fred_df.index = pd.to_datetime(fred_df.index)
fred_df = fred_df.resample("M").last()
fred_df.dropna(inplace=True)
fred_df.head()

/tmp/ipython-input-4-850420165.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fred_df = fred_df.resample("M").last()


,CPI,Unemployment,10Y_Treasury,FedFundsRate,IP_Index
2000-01-31,169.3,4.0,6.66,5.45,91.4092
2000-02-29,170.0,4.1,6.52,5.73,91.7245
2000-03-31,171.0,4.0,6.26,5.85,92.0830
2000-04-30,170.9,3.8,5.99,6.02,92.6659
2000-05-31,171.2,4.0,6.44,6.27,92.9347


In [11]:
#@title WB Data

import wbdata
from datetime import datetime
import pandas as pd

japan_indicators = {
    "NY.GDP.MKTP.KD.ZG": "GDP_Growth",
    "FP.CPI.TOTL.ZG": "Inflation",
    "FR.INR.LEND": "Lending_Rate"
}

df = wbdata.get_dataframe(indicators=japan_indicators, country='JP')

df.index = pd.to_datetime(df.index)

df = df[(df.index >= "2010-01-01") & (df.index <= "2024-12-31")]

df = df.resample("M").ffill()
df.head()

/tmp/ipython-input-11-2718480832.py:17: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample("M").ffill()


,GDP_Growth,Inflation,Lending_Rate
date,,,
2010-01-31,4.097918,-0.728243,1.598333
2010-02-28,4.097918,-0.728243,1.598333
2010-03-31,4.097918,-0.728243,1.598333
2010-04-30,4.097918,-0.728243,1.598333
2010-05-31,4.097918,-0.728243,1.598333


In [13]:
japan_macro = df.copy()
japan_macro.index.name = "Date"

In [15]:
#@title Concatenate

fred_df.index.name = "Date"

macro_df = pd.concat([fred_df, japan_macro], axis=1).dropna()
macro_df.head()

,CPI,Unemployment,10Y_Treasury,FedFundsRate,IP_Index,GDP_Growth,Inflation,Lending_Rate
Date,,,,,,,,
2010-01-31,217.488,9.8,3.73,0.11,89.1897,4.097918,-0.728243,1.598333
2010-02-28,217.281,9.8,3.69,0.13,89.5046,4.097918,-0.728243,1.598333
2010-03-31,217.353,9.9,3.73,0.16,90.1356,4.097918,-0.728243,1.598333
2010-04-30,217.403,9.9,3.85,0.20,90.4607,4.097918,-0.728243,1.598333
2010-05-31,217.290,9.6,3.42,0.20,91.7014,4.097918,-0.728243,1.598333


In [17]:
#@title Download CSV

from google.colab import files

macro_df.to_csv("macro_features.csv")
files.download("macro_features.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>